####MADHUGANDHA CHAVAN
####MT2022058


**Problem Statement**: English to Marathi and Marathi to English bi-directional machine translation using a large language model called indicTrans which is trained on primarily Samanantar dataset and built on top of mBART. Its a transformer, multilingual NMT model.

Tried to address the problem on proverbial translations with indicTrans model.

###Installing Necessary libraries and downloading the models

Here, necessary libraries required for processing, including pandas numpy imported. Model is cloned from repo, along with the important libraries and resource for for Indic NLP.

In [ ]:
import pandas as pd
from google.colab import files
import numpy as np

In [ ]:
# clone the repo for running evaluation
!git clone https://github.com/AI4Bharat/indicTrans.git
%cd indicTrans
# clone requirements repositories
!git clone https://github.com/anoopkunchukuttan/indic_nlp_library.git
!git clone https://github.com/anoopkunchukuttan/indic_nlp_resources.git
!git clone https://github.com/rsennrich/subword-nmt.git
%cd ..

Cloning into 'indicTrans'...
remote: Enumerating objects: 697, done.
remote: Counting objects: 100% (400/400), done.
remote: Compressing objects: 100% (158/158), done.
remote: Total 697 (delta 278), reused 344 (delta 240), pack-reused 297
Receiving objects: 100% (697/697), 2.64 MiB | 3.70 MiB/s, done.
Resolving deltas: 100% (405/405), done.
/content/indicTrans
Cloning into 'indic_nlp_library'...
remote: Enumerating objects: 1362, done.
remote: Counting objects: 100% (143/143), done.
remote: Compressing objects: 100% (48/48), done.
remote: Total 1362 (delta 111), reused 98 (delta 93), pack-reused 1219
Receiving objects: 100% (1362/1362), 9.56 MiB | 13.84 MiB/s, done.
Resolving deltas: 100% (721/721), done.
Cloning into 'indic_nlp_resources'...
remote: Enumerating objects: 139, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 139 (delta 2), reused 2 (delta 0), pack-reused 126
Receiving objects: 100% (139/139), 149.77 MiB |

Installing packages such as mosestokenizer, used for tokenizing Indian language sentences. Installing fairseq for other dependencies such as sacrebleu for calculating BLEU score.   

In [ ]:
! sudo apt install tree
# Install the necessary libraries
!pip install sacremoses pandas mock sacrebleu tensorboardX pyarrow indic-nlp-library
! pip install mosestokenizer subword-nmt
# Install fairseq from source
!git clone https://github.com/pytorch/fairseq.git
%cd fairseq
# !git checkout da9eaba12d82b9bfc1442f0e2c6fc1b895f4d35d
!pip install ./
! pip install xformers
%cd ..

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  tree
0 upgraded, 1 newly installed, 0 to remove and 24 not upgraded.
Need to get 43.0 kB of archives.
After this operation, 115 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal/universe amd64 tree amd64 1.8.0-1 [43.0 kB]
Fetched 43.0 kB in 0s (161 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package tree.
(Reading database ... 122518 files and directories currently installe

In [ ]:
# add fairseq folder to python path
import os
os.environ['PYTHONPATH'] += ":/content/fairseq/"
from fairseq import checkpoint_utils, distributed_utils, options, tasks, utils

###IndicTrans Model

indicTrans is a multilingual NMT model which is trained on a dataset called Samanantar - Largest publicly available parallel corpora for Indian languages.

Link to the documentation on the model:
https://github.com/AI4Bharat/indicTrans

Downloading IndicTrans model: both indic2en (Indian languages to English)  and en2indic(English to Indian Languages)

In [ ]:
# downloading the indic-en model
!wget https://ai4b-public-nlu-nlg.objectstore.e2enetworks.net/indic2en.zip
!unzip indic2en.zip

--2023-05-04 08:04:31--  https://ai4b-public-nlu-nlg.objectstore.e2enetworks.net/indic2en.zip
Resolving ai4b-public-nlu-nlg.objectstore.e2enetworks.net (ai4b-public-nlu-nlg.objectstore.e2enetworks.net)... 101.53.136.18, 164.52.206.154, 164.52.210.96, ...
Connecting to ai4b-public-nlu-nlg.objectstore.e2enetworks.net (ai4b-public-nlu-nlg.objectstore.e2enetworks.net)|101.53.136.18|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4759117228 (4.4G) [application/zip]
Saving to: ‘indic2en.zip’

indic2en.zip        100%[===================>]   4.43G  11.2MB/s    in 6m 42s  

2023-05-04 08:11:15 (11.3 MB/s) - ‘indic2en.zip’ saved [4759117228/4759117228]

Archive:  indic2en.zip
   creating: indic-en/
   creating: indic-en/vocab/
  inflating: indic-en/vocab/bpe_codes.32k.SRC  
  inflating: indic-en/vocab/vocab.SRC  
  inflating: indic-en/vocab/vocab.TGT  
  inflating: indic-en/vocab/bpe_codes.32k.TGT  
   creating: indic-en/final_bin/
  inflating: indic-en/final_bin/prepr

In [ ]:
# downloading the en-indic model
!wget https://ai4b-public-nlu-nlg.objectstore.e2enetworks.net/en2indic.zip
!unzip en2indic.zip

--2023-05-04 08:12:29--  https://ai4b-public-nlu-nlg.objectstore.e2enetworks.net/en2indic.zip
Resolving ai4b-public-nlu-nlg.objectstore.e2enetworks.net (ai4b-public-nlu-nlg.objectstore.e2enetworks.net)... 164.52.210.58, 164.52.210.57, 164.52.210.97, ...
Connecting to ai4b-public-nlu-nlg.objectstore.e2enetworks.net (ai4b-public-nlu-nlg.objectstore.e2enetworks.net)|164.52.210.58|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4811880516 (4.5G) [application/zip]
Saving to: ‘en2indic.zip’

en2indic.zip        100%[===================>]   4.48G  11.3MB/s    in 7m 1s   

2023-05-04 08:19:33 (10.9 MB/s) - ‘en2indic.zip’ saved [4811880516/4811880516]

Archive:  en2indic.zip
   creating: en-indic/
   creating: en-indic/vocab/
  inflating: en-indic/vocab/bpe_codes.32k.SRC  
  inflating: en-indic/vocab/vocab.SRC  
  inflating: en-indic/vocab/vocab.TGT  
  inflating: en-indic/vocab/bpe_codes.32k.TGT  
   creating: en-indic/final_bin/
  inflating: en-indic/final_bin/prepro

In [ ]:
%cd indicTrans

/content/indicTrans


In [ ]:
!pwd

/content/indicTrans


In [ ]:
from indicTrans.inference.engine import Model
en2indic_model = Model(expdir='../en-indic')

Initializing vocab and bpe
Initializing model for translation


In [ ]:
from indicTrans.inference.engine import Model
indic2en_model = Model(expdir='../indic-en')

Initializing vocab and bpe
Initializing model for translation


###Downloading the parallel dataset:


Dataset Preparation:
A dataset of around 7000 sentences is collected and split into train set (5000), dev set (1000), test set(1000).
Source used for dataset collection: https://data.statmt.org/pmindia/v1

In [ ]:
!pwd

/content/indicTrans


In [ ]:
!mkdir /content/dataset
%cd /content/dataset

/content/dataset


In [ ]:
%cd /content/dataset
!wget "https://drive.google.com/uc?id=1zPJ78blSNzX6mNaiFHPuCZq1eNXkslBi&export=download" -O dataset.zip
!unzip dataset.zip

/content/dataset
--2023-05-04 08:22:06--  https://drive.google.com/uc?id=1zPJ78blSNzX6mNaiFHPuCZq1eNXkslBi&export=download
Resolving drive.google.com (drive.google.com)... 172.217.214.100, 172.217.214.102, 172.217.214.138, ...
Connecting to drive.google.com (drive.google.com)|172.217.214.100|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-0k-a0-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/bgobetdj8r6mrg0v2vtjpp117i40863q/1683188475000/10066636681463159322/*/1zPJ78blSNzX6mNaiFHPuCZq1eNXkslBi?e=download&uuid=0411d02a-db98-463a-aad2-c2ab99e86d1b [following]
--2023-05-04 08:22:07--  https://doc-0k-a0-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/bgobetdj8r6mrg0v2vtjpp117i40863q/1683188475000/10066636681463159322/*/1zPJ78blSNzX6mNaiFHPuCZq1eNXkslBi?e=download&uuid=0411d02a-db98-463a-aad2-c2ab99e86d1b
Resolving doc-0k-a0-docs.googleusercontent.com (doc-0k-a0-docs.googleusercontent.com)... 

Fetching test set from entire dataset to test the base models (both en2indic and indic2en) on it.

In [ ]:
f=open("/content/dataset/dataset_finetune/test/test.en","r")
test_en_list_full=[]
for x in f:
  test_en_list_full.append(x)

test_en_list=test_en_list_full

In [ ]:
test_en_list[:5]

['Now, the next decade is crucial.\n',
 'In an environment where we seek stability, sustainable development and prosperity.\n',
 'BRICS leadership will be crucial in driving this transformation.\n',
 'If we as BRICS can set the agenda in these areas, the world will call this its Golden Decade.\n',
 'In our outreach segment with emerging markets tomorrow.\n']

In [ ]:
f=open("/content/dataset/dataset_finetune/test/test.mr","r")
test_mr_list_full=[]
for x in f:
  test_mr_list_full.append(x)

test_mr_list=test_mr_list_full

In [ ]:
test_mr_list[:5]

['आता यापुढचे दशक अतिशय महत्त्वाचे आहे.\n',
 'स्थैर्य, शाश्वत विकास आणि समृद्धी असलेले वातावरण निर्माण व्हावे अशी अपेक्षा आपण करत आहोत.\n',
 'ब्रिक्स नेतृत्व या परिवर्तनाच्या दिशेने वाटचाल करण्यासाठी महत्त्वाचे ठरेल.\n',
 'जर आपण ब्रिक्स या नात्याने या क्षेत्रांच्या संदर्भात एक धोरण निश्चित करू शकलो तर जग या दशकाला सुवर्ण दशक समजू लागेल.\n',
 'उदयोन्मुख बाजारपेठांसोबत आपला उद्या विचारविनिमय होणार आहे.\n']

###Tokenizing both EN and Mr test datasets:
Tokenizing ground truths as well as predictions using indic_tokenize of indic-nlp-library
indic_tokenize is tokenizer module for Indian languages and it supports trivial tokenizer which is simple punctuation based tokenizer.

In [ ]:
!pip install indic-nlp-library
from indicnlp.tokenize import indic_tokenize

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
tokenized_test_mr_list=[]
for i in range(len(test_mr_list)):
    tokenized_words=indic_tokenize.trivial_tokenize(test_mr_list[i])
    tokenized_words.pop()
    tokenized_test_mr_list.append([tokenized_words])

tokenized_test_en_list=[]
for i in range(len(test_en_list)):
  tokenized_words=indic_tokenize.trivial_tokenize(test_en_list[i])
  tokenized_words.pop()
  tokenized_test_en_list.append([tokenized_words])


####Predictions using indicTrans model:

Infering using indicTrans model from both the directions (Translating sentences from English to Marathi and Marathi to English)

In [ ]:
BATCH_SIZE = 2
NUM_BATCHES = len(test_en_list)//BATCH_SIZE + 1
predictions_mr=[]
for i in range(NUM_BATCHES):
  en_sents_batch = test_en_list[BATCH_SIZE*i : BATCH_SIZE*(i+1)]
  predictions_mr.extend(en2indic_model.batch_translate(en_sents_batch,'en','mr'))

100%|██████████| 2/2 [00:00<00:00, 769.24it/s]
0it [00:00, ?it/s]


In [ ]:
BATCH_SIZE = 2
NUM_BATCHES = len(test_mr_list)//BATCH_SIZE + 1
predictions_en=[]
for i in range(NUM_BATCHES):
  mr_sents_batch = test_mr_list[BATCH_SIZE*i : BATCH_SIZE*(i+1)]
  predictions_en.extend(indic2en_model.batch_translate(mr_sents_batch,'mr','en'))

100%|██████████| 2/2 [00:00<00:00, 1745.44it/s]
0it [00:00, ?it/s]


###Calculating BLEU Score on test dataset:
Both NLTK and Sacrebleu provide functions to calculate BLEU score, but they have different tokenization rules. Scarebleu takes every sentence in a corpus as a separate list. This function strictly takes detokenized sentences as an input.
 Whereas NLTK provides corpus_bleu and sentence_bleu modules to calculate corpus level and sentence level BLEU scores. These two functions strictly take tokenized sentences as an input.
There can be multiple possible translations of a sentence in Marathi for a sentence in English and vice versa. Hence, for every sentence in a test set, reference that will be used will be a list of possible directions.

Based on predictions and refernces, BLEU score is calculated on the test set that has been curated and it turned out to be:
1. For English to Marathi Translations: 21.36
2. For Marathi to English Translations: 65.01

In [ ]:
ref_mr_list=[[x] for x in test_mr_list]
ref_en_list=[[x] for x in test_en_list]

In [ ]:
from sacrebleu.metrics import BLEU

In [ ]:
bleu = BLEU()

In [ ]:
ref_mr_list[:5]

[['आता यापुढचे दशक अतिशय महत्त्वाचे आहे.\n'],
 ['स्थैर्य, शाश्वत विकास आणि समृद्धी असलेले वातावरण निर्माण व्हावे अशी अपेक्षा आपण करत आहोत.\n'],
 ['ब्रिक्स नेतृत्व या परिवर्तनाच्या दिशेने वाटचाल करण्यासाठी महत्त्वाचे ठरेल.\n'],
 ['जर आपण ब्रिक्स या नात्याने या क्षेत्रांच्या संदर्भात एक धोरण निश्चित करू शकलो तर जग या दशकाला सुवर्ण दशक समजू लागेल.\n'],
 ['उदयोन्मुख बाजारपेठांसोबत आपला उद्या विचारविनिमय होणार आहे.\n']]

In [ ]:
predictions_mr[:5]

['आता येणारे दशक महत्वाचे आहे.',
 'अशा वातावरणात जिथे आम्हाला स्थिरता, शाश्वत विकास आणि समृद्धी हवी आहे.',
 'हे परिवर्तन घडवून आणण्यासाठी ब्रिक्स नेतृत्व महत्वाचे ठरेल.',
 'जर आपण ब्रिक्स म्हणून या क्षेत्रांमध्ये कार्यक्रम निश्चित करू शकलो तर जग या दशकाला सुवर्ण दशक म्हणेल.',
 'उद्या उदयोन्मुख बाजारपेठांपर्यंत पोहोचण्यासाठी आम्ही प्रयत्नशील आहोत.']

In [ ]:
bleu.corpus_score(predictions_mr, ref_mr_list)

BLEU = 21.36 100.0/20.0/12.5/8.3 (BP = 1.000 ratio = 1.000 hyp_len = 6 ref_len = 6)

In [ ]:
bleu.corpus_score(predictions_en, ref_en_list)

BLEU = 44.18 100.0/57.1/33.3/20.0 (BP = 1.000 ratio = 1.000 hyp_len = 8 ref_len = 8)

###Translating DEV set for checking for possible improvments in translation quality:

Link to that CSV file:
https://docs.google.com/spreadsheets/d/1PjejrqBwWB82KuTbXBuIfXZTkjWvik9X/edit?usp=sharing&ouid=109597389439636620384&rtpof=true&sd=true

In [ ]:
f=open("/content/dataset/dataset_finetune/dev/dev.en","r")
dev_en_list_full=[]
for x in f:
  dev_en_list_full.append(x)

dev_en_list=dev_en_list_full

f=open("/content/dataset/dataset_finetune/dev/dev.mr","r")
dev_mr_list_full=[]
for x in f:
  dev_mr_list_full.append(x)

dev_mr_list=dev_mr_list_full

In [ ]:
ref_dev_mr_list=[[x] for x in dev_mr_list]
ref_dev_en_list=[[x] for x in dev_en_list]

In [ ]:
BATCH_SIZE = 2
NUM_BATCHES = len(dev_mr_list)//BATCH_SIZE + 1
predictions_dev_en=[]
for i in range(NUM_BATCHES):
  dev_mr_sents_batch = dev_mr_list[BATCH_SIZE*i : BATCH_SIZE*(i+1)]
  predictions_dev_en.extend(indic2en_model.batch_translate(dev_mr_sents_batch,'mr','en'))

100%|██████████| 2/2 [00:00<00:00, 3240.10it/s]
0it [00:00, ?it/s]


####Creating a CSV file on dev set and arranging the parallel sentences in ascending order of their BLEU scores to analyse more:
Calculating sentence level BLEU scores on dev set to evaluate quality and creating a CSV file for it to analyse parallel sentences in ascending order of their BLEU scores. Lower the BLEU score, worse is the quality of the translations.


In [ ]:
en_sentence_bleu_rows = []

for i in range(len(predictions_dev_en)):
  bleu_score = bleu.sentence_score(predictions_dev_en[i], ref_dev_en_list[i])
  en_sentence_bleu_rows.append({
      "en_sent": str(dev_mr_list[i][:-1]),
      "mr_ref":  str(ref_dev_en_list[i][0][:-1]),
      "mr_pred": str(predictions_dev_en[i][:-1]),
      "bleu_score": bleu_score.score
  })

bleu_df = pd.DataFrame(en_sentence_bleu_rows)

display(bleu_df.head())

bleu_df.to_csv("/content/en_bleu_scores.csv", index=False)

,en_sent,mr_ref,mr_pred,bleu_score
0,"जनतेला संबोधित करताना पंतप्रधान म्हणाले की, अं...","Addressing the public meeting, the Prime Minis...","Addressing the gathering, the Prime Minister s...",52.968650
1,त्यांना उत्तर प्रदेश आणि बिहार दरम्यान धावणाऱ्...,He said the train will make it easier for peop...,This express between Uttar Pradesh and Bihar w...,48.550645
2,"मोदी पुढे म्हणाले की, पशुपालन व्यवसायातील गुजर...",Neem coating of urea helped farmers and stoppe...,The Prime Minister said that Gujarats strides ...,5.254053
3,"ते म्हणाले की, वाराणसीत नुकताच अशाच प्रकारच्या...",Shri Narendra Modi said that Gujarat’s strides...,He said that a similar Mela had been organised...,3.741944
4,नीम आवरण असलेल्या युरिया मुळे शेतक-यांना मदत झ...,He said that he had asked the Chief Minister o...,Neem Coated Urea has helped the farmers and el...,1.187380


In [ ]:
sample_sentences = ["दुष्काळात तेरावा महिना", "शब्दापेक्षा कृती जास्त बोलकी असते"]

indic2en_model.batch_translate(sample_sentences,'mr','en')

100%|██████████| 2/2 [00:00<00:00, 3930.93it/s]


['13th month of drought', 'Actions speak louder than words']

##Problems with the translations using existing model:

While analysing different translations generated from both the models, it was observed that while translating a proverb from Marathi to English, it translates it to the literal meaning of the proverb rather than capturing the semantic meaning.

For example, 'एका हाताने टाळी वाजत नसते' translates to 'No clapping with one hand' wheras its actual semantic meaning is 'It takes two to make quarrel'.

For English to Marathi proverb translation, it performs well and translates it to the closest marathi proverb/meaning possible.


In [ ]:
mr_sents=['चकावते ते सोने नसते','एक ना धड भाराभर चिंध्या','दुष्काळात तेरावा महिना',
          'माकडाच्या हाती कोलीत','नाचता येईना अंगण वाकडे','अंथरूण पाहून पाय पसरावेत',
          'उथळ पाण्याला खळखळाट फार','बळी तो कान पिळी','एका हाताने टाळी वाजत नसते','इकडे आड तिकडे विहीर',
          'वरातीमागून घोडे','अतिपरिचयात अवज्ञा','वाहत्या गंगेत हात धुवून घेणे','बडा घर पोकळ वासा']


indic2en_model.batch_translate(mr_sents, 'mr', 'en')

100%|██████████| 14/14 [00:00<00:00, 11201.88it/s]


['Gold is not a metal',
 'Not a single scratch',
 '13th month of drought',
 "In the monkey's hand",
 'Unable to dance',
 'Spread your feet by looking at the bed',
 'Very deep water',
 'The victim has a yellow ear',
 'No clapping with one hand',
 'A well here and there',
 'Horses behind the chariot',
 'Over-identity denial',
 'Washing hands with soap',
 'The big house']

In [ ]:
en_sents=['Its an apple',
            "It was raining cats and dogs  ",
            'Are you even listening to me', 'Try and always remains happy', 'Can you please move that chair to me','All’s well that ends well','Action speak louder than word']

en_sents=['All’s well that ends well','Action speak louder than word','Beggars can’t be choosers.','Barking dogs seldom bite']
en2indic_model.batch_translate(en_sents, 'en', 'mr')



100%|██████████| 4/4 [00:00<00:00, 3555.25it/s]


['सर्व काही चांगले आहे ज्याचा शेवट',
 'शब्दांपेक्षा कृती जास्त बोलली जाते',
 'भिकारी हे निवडकर्ते असू शकत नाहीत.',
 'भुंकणारे कुत्रे क्वचितच चावतात']

### Preprocessing and Finetuning:

Link to the finetuning notebook is as follows:

Finetuning English-Marathi model:
https://colab.research.google.com/drive/1ERdAjaxGJPz4BwTvHKvk_xHSPTPcGbCc?usp=sharing

Finetuning Marathi-English model:
https://colab.research.google.com/drive/1qyr-cYQqbcAyiDiv_PDOJ-DXLWCygzqU?usp=sharing


**Sources used for dataset collection and curation proverbs are as follows:**

Parallel corpus is created for marathi proverbs in such a way that if an equivalent English proverb is there, then that will be there in the corpus and if not, then semantic meaning of the proverb is present.

Sources used for creating the proverbs dataset:

1.   https://programmar4u.blogspot.com/2021/09/proverb.html
2.   https://learning-marathi.blogspot.com/2020/03/18-marathi-proverbs.html
3.   https://www.webshodhinmarathi.com/common-english-proverb-with-their-marathi-meaning/
4.   https://englishforlearner.com/translations-of-english-proverbs-to-marathi/

From all these resources, dataset containing 250 pairs of proverbs and their translations is created.

Link to the curated parallel corpus of only proverbs:
https://drive.google.com/file/d/1S4M9D1_9hyDi7U98asTbnxGC3Zyar-Yj/view?usp=sharing

Paper taken as an inspiration for creating a proverbial based second dataset:


*   Proverbial translations is an important ongoing research problem.
*   A paper from 2022 refers to the same problems but with idioms. Link to the paper: https://arxiv.org/pdf/2210.04545.pdf
*   It is trained on a set containing idioms merged with different sentences.
*   Inspired from the same thought, here is a curated dataset of having 2 sentences for each proverb. One containing the semantic meaning of the proverb and other one refering to literal meaning.
Link to the curated dataset: https://drive.google.com/file/d/1RLT6GojPN9oSr9YUmcJZ1aeM-az721Zb/view?usp=sharing
*   Then dataset is incorporated with the finetune dataset created earlier, and with that dataset finetuning is done ultimately. \
*   Dataset is upsampled fro 5 times and merged with the finetuning dataset created earlier.


